# Focus on LBM MNAR model

### 0- Library importation 

In [1]:
import torch
import numpy as np
from torch import nn
from fcts.utils import reparametrized_expanded_params, d2_DL3_XO, init_random_params

### 1- Load dataset & parameters

a- Dataset

In [2]:
#votes: matrix gathering votes for several laws and politicians (1: positive, 0: missing/abstention, -1: negative)
votes = np.loadtxt("data_parliament/votes.txt",delimiter=";").astype(int)

b- Parameter initialization

In [3]:
# Dataset size
n1, n2 = votes.shape
nq = 3 # COMPLETE
nl = 5 #COMPLETE

#torch parameters
device = 'mps' #put 'cuda', 'cpu' or 'mps' (for Mac)
device2 = 'mps' #put None, 'cuda' or 'mps' (for Mac) 

# gamma and theta 
vector_of_parameters = torch.tensor(init_random_params(n1, n2, nq, nl), requires_grad=True, device=device, dtype=torch.float32)
lengamma = (4 * n1+ 4 * n2+ (n1 * (nq - 1))+ (n2 * (nl - 1)))
variationnal_params = vector_of_parameters[:lengamma].clone()
model_params = vector_of_parameters[lengamma:].clone()

#indices
indices_p = np.argwhere(votes == 1) #argwhere: matrix with couples (row,column) with 1 values 
indices_n = np.argwhere(votes == -1) #idem with -1
indices_zeros = np.argwhere(votes == 0) #idem with 0 

### 2- Computation of the criterion: 

$J(\gamma, \theta) = H(q) + E_q[log(p(X^o, Y_1, Y_2, A, B, C, D, \gamma ))]$


#### a- Entropy 

$H(q) = - \Sigma_{ik} \tau_{ik}^{(Y_1)}log(\tau_{ik}^{(Y_1)}) - \Sigma_{jl} \tau_{jl}^{(Y_2)}log(\tau_{jl}^{(Y_2)}) + \frac{1}{2} \Sigma_i log(2\pi e \rho_i^{(A)}) + \frac{1}{2} \Sigma_i log(2\pi e \rho_i^{(B)})+ \frac{1}{2} \Sigma_j log(2\pi e \rho_j^{(C)})+ \frac{1}{2} \Sigma_j log(2\pi e \rho_j^{(D)})$

In [4]:
def entropy_rx(rho_a, rho_b, rho_p, rho_q, tau_1, tau_2):
    return (1/2* (2 * n1 + 2 * n2)* 
        (torch.log(torch.tensor(2 * np.pi, dtype=torch.float32, device=device))+ 1)
        + 1 / 2 * torch.sum(torch.log(rho_a))
        + 1 / 2 * torch.sum(torch.log(rho_b))
        + 1 / 2 * torch.sum(torch.log(rho_p))
        + 1 / 2 * torch.sum(torch.log(rho_q))
        - torch.sum(tau_1 * torch.log(tau_1))
        - torch.sum(tau_2 * torch.log(tau_2)))

#### b- Expectation with respect to the variational distribution 

$E_q[log(p(X^o, Y_1, Y_2, A, B, C, D, \gamma ))] = E_{q_{\gamma}}[log(p(Y_1))] + E_{q_{\gamma}}[log(p(Y_2))] + E_{q_{\gamma}}[log(p(A))] + E_{q_{\gamma}}[log(p(B))] + E_{q_{\gamma}}[log(p(C))] + E_{q_{\gamma}}[log(p(D))]$

$E_{q_{\gamma}}[log(p(Y_1))] = \Sigma_{ik} \tau_{ik}^{(Y_1)}log(\alpha_k)$

In [10]:
def expectation_loglike_Y1(tau_1, alpha_1):
    return tau_1.sum(0) @ torch.log(alpha_1)

$E_{q_{\gamma}}[log(p(Y_2))] = \Sigma_{jl} \tau_{jl}^{(Y_2)}log(\beta_l)$

In [11]:
def expectation_loglike_Y2(tau_2, alpha_2):
    return tau_2.sum(0) @ torch.log(alpha_2).t()

$E_{q_{\gamma}}[log(p(I))] = -\frac{n_1}{2}log(2\pi) - \frac{n_1}{2}log(\sigma_I^2) - \frac{1}{2\sigma_I^2} \Sigma_{i} (((\nu_i^{(I)})^2 + \rho_i^{(I)}))$ for $I \in {A, B}$

In [6]:
def expectation_loglike_A(nu_a, rho_a, sigma_sq_a):
        return -n1 / 2 * (torch.log(torch.tensor(2 * np.pi, dtype=torch.float32, device=device))+ torch.log(sigma_sq_a)) - 1 / (2 * sigma_sq_a) * torch.sum(rho_a + nu_a ** 2)

def expectation_loglike_B(nu_b, rho_b, sigma_sq_b):
    return -n1 / 2 * (torch.log(torch.tensor(2 * np.pi, dtype=torch.float32, device=device))+ torch.log(sigma_sq_b)) - 1 / (2 * sigma_sq_b) * torch.sum(rho_b + nu_b ** 2)

$E_{q_{\gamma}}[log(p(J))] = -\frac{n_2}{2}log(2\pi) - \frac{n_2}{2}log(\sigma_J^2) - \frac{1}{2\sigma_J^2} \Sigma_{j} (((\nu_j^{(J)})^2 + \rho_j^{(J)}))$ for $J \in {C, D}$

In [ ]:
def expectation_loglike_P(nu_p, rho_p, sigma_sq_p):
    return -n2 / 2 * (torch.log(torch.tensor(2 * np.pi, dtype=torch.float32, device=device))+ torch.log(sigma_sq_p)) - 1 / (2 * sigma_sq_p) * torch.sum(rho_p + nu_p ** 2)

def expectation_loglike_Q(nu_q, rho_q, sigma_sq_q):
    return -n2 / 2 * (torch.log(torch.tensor(2 * np.pi, dtype=torch.float32, device=device))+ torch.log(sigma_sq_q)) - 1 / (2 * sigma_sq_q) * torch.sum(rho_q + nu_q ** 2)

$E_{q_\gamma}[log(p(X^o|Y_1, Y_2, A, B, C, D))] = \Sigma_{(kl, ij, X_{ij}^o=1)}\tau_{ij}^{(Y_1)}\tau_{jl}^{(Y_2)}E_{q_{\gamma}}[log(p_1)] + \Sigma_{(kl, ij, X_{ij}^o=0)}\tau_{ij}^{(Y_1)}\tau_{jl}^{(Y_2)}E_{q_{\gamma}}[log(p_0)] + \Sigma_{(kl, ij, X_{ij}^o=NA)}\tau_{ij}^{(Y_1)}\tau_{jl}^{(Y_2)}E_{q_{\gamma}}[log(1- p_0 -p_1)]$

Where: 

 $E_{q_{\gamma}}[f_h(A_i + C_j, B_i + D_j)] ≈ f_h(\nu_i^{(A)} + \nu_j^{(C)}, \nu_i^{(B)} + \nu_j^{(D)}) + \frac{1}{2}(\rho_i^{(A)} + \rho_j^{(C)})\frac{\partial^2f_h(\nu_i^{(A)} + \nu_j^{(C)}, \nu_i^{(B)} + \nu_j^{(D)})}{\partial(\nu_i^{(A)} + \nu_j^{(C)})^2} + \frac{1}{2}(\rho_i^{(B)} + \rho_j^{(D)})\frac{\partial^2f_h(\nu_i^{(A)} + \nu_j^{(C)}, \nu_i^{(B)} + \nu_j^{(D)})}{\partial(\nu_i^{(B)} + \nu_j^{(B)})^2} $ for $h \in \{1, 0, NA\}$
 
 
 and 
 
 $f_1(x,y) = log(\pi_{kl}expit(\mu + x + y))$

$f_0(x,y) = log((1 - \pi_{kl})expit(\mu + x - y))$

$f_{NA}(x,y) = log(1 - \pi_{kl}expit(\mu + x +y) - (1 - \pi_{kl})expit(\mu + x -y))$

In [ ]:
def expectation_loglike_X_cond_ABPY1Y2(
        mu_un,nu_a,nu_b,nu_p,nu_q,rho_a,rho_b,rho_p,rho_q,tau_1,tau_2,pi):
        i_p_one = indices_p[:, 0] # takes rows Xij=1
        i_m_one = indices_n[:, 0] #takes rows Xij=0
        i_zeros = indices_zeros[:, 0] #takes rows Xij=NA
        j_p_one = indices_p[:, 1] # cols Xij=1
        j_m_one = indices_n[:, 1] #cols Xij=0
        j_zeros = indices_zeros[:, 1] #cols Xij=NA

        ## Positives: Xij= 1 ### 
        xp = nu_a[i_p_one].flatten() + nu_p[:, j_p_one].flatten() #for A & C
        yp = nu_b[i_p_one].flatten() + nu_q[:, j_p_one].flatten() #for B & D

        sig_p = torch.sigmoid(mu_un+ nu_a[i_p_one]+ nu_b[i_p_one]+ nu_p[:, j_p_one].t()+ nu_q[:, j_p_one].t())
        der2_sig_p = (-sig_p * (1 - sig_p)).flatten()
        sum_var_p = (rho_a[i_p_one].flatten()+ rho_p[:, j_p_one].flatten()+ rho_b[i_p_one].flatten()+ rho_q[:, j_p_one].flatten())

        # f_{1}
        f = lambda x, y: torch.log(pi.view(1, nq, nl) * torch.sigmoid(mu_un + x + y).view(-1, 1, 1))
        
        # Taylor development (1)
        expectation_taylor_p = (tau_1[i_p_one].view(-1, nq, 1)
            * tau_2[j_p_one].view(-1, 1, nl)
            * (f(xp, yp) + 0.5 * (der2_sig_p * sum_var_p).view(-1, 1, 1))).sum()

        ### Negatives: Xij= -1 ###
        xn = nu_a[i_m_one].flatten() + nu_p[:, j_m_one].flatten()
        yn = nu_b[i_m_one].flatten() + nu_q[:, j_m_one].flatten()
        
        sig_m = torch.sigmoid(mu_un+ nu_a[i_m_one]- nu_b[i_m_one]+ nu_p[:, j_m_one].t()- nu_q[:, j_m_one].t())
        der2_sig_m = -(sig_m * (1 - sig_m)).flatten()
        sum_var_m = (rho_a[i_m_one].flatten()+ rho_p[:, j_m_one].flatten()+ rho_b[i_m_one].flatten()+ rho_q[:, j_m_one].flatten())
        
        #f_{0}
        f = lambda x, y: torch.log((1 - pi).view(1, nq, nl)* torch.sigmoid(mu_un + x - y).view(-1, 1, 1))

        # Taylor development (0)
        expectation_taylor_m = (tau_1[i_m_one].view(-1, nq, 1)* tau_2[j_m_one].view(-1, 1, nl)
            * (f(xn, yn) + 0.5 * (der2_sig_m * sum_var_m).view(-1, 1, 1))).sum()

        ### Zeros: Xij= 0 ###

        # f_{NA}
        f = lambda x, y: torch.log(1- pi.view(1, nq, nl) * torch.sigmoid(mu_un + x + y).view(-1, 1, 1)- (1 - pi.view(1, nq, nl))* torch.sigmoid(mu_un + x - y).view(-1, 1, 1))
        xz = nu_a[i_zeros].flatten() + nu_p[:, j_zeros].flatten()
        yz = nu_b[i_zeros].flatten() + nu_q[:, j_zeros].flatten()

        if device2:
            der_x = d2_DL3_XO(xz.view(-1, 1, 1).to(device2),yz.view(-1, 1, 1).to(device2),mu_un.to(device2),pi.view(1, nq, nl).to(device2),"x",).to(device)
        else:
            der_x = d2_DL3_XO(xz.view(-1, 1, 1),yz.view(-1, 1, 1),mu_un,pi.view(1, nq, nl),"x",)

        der_y = d2_DL3_XO(xz.view(-1, 1, 1),yz.view(-1, 1, 1),mu_un,pi.view(1, nq, nl),"y",)

        tau_12_ij = tau_1[i_zeros].view(-1, nq, 1) * tau_2[j_zeros].view(-1, 1, nl)
        
        # Taylor development (NA)
        expectation_taylor_zeros = (tau_12_ij* (f(xz, yz)+ 0.5
                * (der_x* (rho_a[i_zeros].flatten() + rho_p[:, j_zeros].flatten()).view(-1, 1, 1)
                    + der_y* (rho_b[i_zeros].flatten() + rho_q[:, j_zeros].flatten()).view(-1, 1, 1)))).sum()



        ### Final expectation ###
        expectation = (expectation_taylor_p+ expectation_taylor_m+ expectation_taylor_zeros)
        
        return (expectation if expectation < 0 else torch.tensor(np.inf, device=device))


Final computation of criterion: 

$J(\gamma, \theta) = H(q) + E_q[log(p(X^o, Y_1, Y_2, A, B, C, D, \gamma ))]$


In [13]:
def criteria(variationnal_params, model_params):
        (nu_a,rho_a,nu_b,rho_b,nu_p,rho_p,nu_q,rho_q,tau_1,tau_2,mu_un,sigma_sq_a,sigma_sq_b,sigma_sq_p,sigma_sq_q,alpha_1,alpha_2,
            pi) = reparametrized_expanded_params(torch.cat((variationnal_params, model_params)),n1,n2,nq,nl,device,)
        if torch.any(tau_1.sum(dim=0) < 0.5):
            print("One empty row class, algo stoped")
            return torch.tensor(np.nan, device=device)
        if torch.any(tau_2.sum(dim=0) < 0.5):
            print("One empty column class, algo stoped")
            return torch.tensor(np.nan, device=device)
        
        
        expectation = (entropy_rx(rho_a, rho_b, rho_p, rho_q, tau_1, tau_2)
            + expectation_loglike_A(nu_a, rho_a, sigma_sq_a)[0]
            + expectation_loglike_B(nu_b, rho_b, sigma_sq_b)[0]
            + expectation_loglike_P(nu_p, rho_p, sigma_sq_p)[0]
            + expectation_loglike_Q(nu_q, rho_q, sigma_sq_q)[0]
            + expectation_loglike_Y1(tau_1, alpha_1)
            + expectation_loglike_Y2(tau_2, alpha_2)
            + expectation_loglike_X_cond_ABPY1Y2(mu_un,nu_a,nu_b,nu_p,nu_q,rho_a,rho_b,rho_p,rho_q,tau_1,tau_2,pi)
        )
        
        #We return -expectation to transform the maximization problem into a minimization problem 
        return -expectation